In [1]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

## 딥러닝 모델

In [2]:
class NeuralNet(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet, self).__init__()
        self.mlp1 = layers.Dense(hidden_1, activation=tf.nn.relu)
        self.mlp2 = layers.Dense(hidden_2, activation="relu") # 둘 다 사용 가능
        self.mlp3 = layers.Dense(num_classes) # actiavtion="softmax"

    def call(self, x):
        print('x:',x.shape)
        x = self.mlp1(x)
        print('x2:',x.shape)
        x = self.mlp2(x)
        print('x3:',x.shape)
        x = self.mlp3(x)
        print('x4:',x.shape)
        x = tf.nn.softmax(x)
        return x

In [3]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

In [4]:
model = NeuralNet(hidden_1, hidden_2, num_classes)

2021-09-19 19:07:02.079342: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-19 19:07:02.079559: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
data = tf.random.normal(shape=(1,28*28), mean=0., stddev=1.)
print('data:', data.shape)
print('-'*40)

pred = model(data)
print('pred:', pred)
print('-'*40)

y_hat = tf.argmax(pred, 1)
print('y hat:', y_hat)

data: (1, 784)
----------------------------------------
x: (1, 784)
x2: (1, 128)
x3: (1, 256)
x4: (1, 10)
pred: tf.Tensor(
[[0.13949569 0.13945116 0.12087785 0.28384194 0.08963875 0.02253515
  0.01823299 0.0438124  0.02067833 0.12143576]], shape=(1, 10), dtype=float32)
----------------------------------------
y hat: tf.Tensor([3], shape=(1,), dtype=int64)


### layers.Dense

In [6]:
x = tf.random.normal(shape=(3,28,28), mean=0., stddev=1.)

Without activation function

In [7]:
fc_layer = layers.Dense(100)
x_after_fc = fc_layer(tf.reshape(x,(-1,28*28)))
print('x_after_fc:',x_after_fc.shape)
print(x_after_fc[0][:10])

x_after_fc: (3, 100)
tf.Tensor(
[ 2.0233107  -2.6097698  -0.73857266  1.1991029   2.8984256   0.18412846
 -0.03651004 -1.3617487  -0.09439725  0.9603974 ], shape=(10,), dtype=float32)


With activation function

In [8]:
fc_layer = layers.Dense(100, activation='relu')
x_after_fc = fc_layer(tf.reshape(x,(-1,28*28)))
print('x_after_fc:',x_after_fc.shape)
print(x_after_fc[0][:10])

x_after_fc: (3, 100)
tf.Tensor(
[0.03649342 0.5683936  1.1797311  0.56895393 0.         1.9251724
 0.         0.         1.4246778  0.87268186], shape=(10,), dtype=float32)


### Softmax

In [9]:
x = tf.random.normal(shape=(1,10), mean=0., stddev=1.)
x

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 0.10982104, -1.277446  ,  0.48827   , -0.8570649 , -1.3606479 ,
         0.5870613 ,  2.6384974 , -0.3602169 ,  0.00689749, -0.78907144]],
      dtype=float32)>

In [10]:
pred = tf.nn.softmax(x)
print('pred:',pred)
print('sum of pred:',tf.reduce_sum(pred))

pred: tf.Tensor(
[[0.05153955 0.01287236 0.07524869 0.0195987  0.0118447  0.08306221
  0.6461462  0.03221111 0.04649877 0.02097763]], shape=(1, 10), dtype=float32)
sum of pred: tf.Tensor(0.9999999, shape=(), dtype=float32)


### tf.keras.Sequential

In [11]:
class NeuralNet(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = layers.Dense(hidden_1, activation=tf.nn.relu)
        self.fc2 = layers.Dense(hidden_2, activation=tf.nn.relu)
        self.out = layers.Dense(num_classes)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        x = tf.nn.softmax(x)
        return x

In [12]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

model = NeuralNet(hidden_1, hidden_2, num_classes)

In [13]:
class NeuralNet2(Model):
    def __init__(self, hidden_1, hidden_2, num_classes):
        super(NeuralNet2, self).__init__()
        sequential = tf.keras.Sequential([
            layers.Dense(hidden_1, activation=tf.nn.relu),
            layers.Dense(hidden_2, activation=tf.nn.relu),
            layers.Dense(num_classes, activation=tf.nn.softmax),
        ])

    def call(self, x):
        x = sequential(x)
        return x

In [14]:
hidden_1 = 128
hidden_2 = 256
num_classes = 10

model = NeuralNet2(hidden_1, hidden_2, num_classes)

### Sequential + add

In [15]:
model = tf.keras.Sequential()

In [16]:
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### Compile

In [17]:
model.compile(loss='categorical_crossentropy', # labels가 one-hot일 경우 사용
              optimizer='sgd',
              metrics=['accuracy'])

In [18]:
model.compile(loss='sparse_categorical_crossentropy', # labels가 integer일 경우 사용
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), # learning rate 설정
              metrics=['accuracy'])